## 1. Initialization/Preliminaries

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import collections
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
import sklearn.svm as svm
from sklearn.model_selection import GridSearchCV
from scipy.ndimage.measurements import label


from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split


## 2. Parameters

In [2]:
features_params = {
    'color_space' : 'YCrCb',    # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    'spatial_size' : (16, 16),  # Spatial binning dimensions
    'hist_bins' : 32,           # Number of histogram bins
    'orient' : 11,              # HOG orientations
    'pix_per_cell' : 16,        # HOG pixels per cell
    'cell_per_block' : 2,       # HOG cells per block
    'hog_channel' : 'ALL',      # Can be 0, 1, 2, or 'ALL'
    'spatial_feat' : True,      # Spatial features on or off
    'hist_feat' : True,         # Histogram features on or off
    'hog_feat' : True,          # HOG features on or off
}

y_start_stop = (400, None)

## 3. Utility functions

Miscelleous functions (mostly graphics oriented).

In [3]:

color_map = { ('RGB', 'HSV') : cv2.COLOR_RGB2HSV,
              ('RGB', 'LUV') : cv2.COLOR_RGB2LUV,
              ('RGB', 'HLS') : cv2.COLOR_RGB2HLS,
              ('RGB', 'YUV') : cv2.COLOR_RGB2YUV,
              ('RGB', 'YCrCb') : cv2.COLOR_RGB2YCrCb,
              ('RGB', 'BGR') : cv2.COLOR_RGB2BGR,
              ('RGB', 'LAB') : cv2.COLOR_RGB2LAB,
             
              ('BGR', 'HSV') : cv2.COLOR_BGR2HSV,
              ('BGR', 'LUV') : cv2.COLOR_BGR2LUV,
              ('BGR', 'HLS') : cv2.COLOR_BGR2HLS,
              ('BGR', 'YUV') : cv2.COLOR_BGR2YUV,
              ('BGR', 'YCrCb') : cv2.COLOR_BGR2YCrCb,
              ('BGR', 'RGB') : cv2.COLOR_BGR2RGB,
              ('BGR', 'LAB') : cv2.COLOR_BGR2LAB,
            }

def convert_colorspace(image, source_color, dest_color):
    """
    Converts the image from one color space to another (creates a new image)
    If the source color and dest color are the same, the image is just copied.
    """
    if source_color == dest_color:
        return np.copy(image)
    return cv2.cvtColor(image, color_map[(source_color, dest_color)])


# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=8):
    # Make a copy of the image
    #imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(img, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return img


def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,255,0), 6)
    # Return the image
    return img




## 4. Feature extraction

In [4]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
def extract_features(imgs, color_space='RGB',
                     spatial_size=(32, 32),
                     hist_bins=32,
                     orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0,
                     spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)

        # Image assumed to be RGB since we're reading a PNG from mpimg
        feature_image = convert_colorspace(image, 'RGB', color_space)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    

## 5. Training

In [5]:
def train_svc(color_space='RGB', 
              spatial_size=(32, 32),
              hist_bins=32, hist_range=(0, 256),
              orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0,
              spatial_feat=True, hist_feat=True, hog_feat=True):

    cars = []
    notcars = []

    cars_directory = '/Users/kennt/classes/udacity/P5/vehicles/*/*.png'
    nocars_directory = '/Users/kennt/classes/udacity/P5/non-vehicles/*/*.png'

    #cars_directory = '/Users/kennt/classes/udacity/small/vehicles_smallset/*/*.jpeg'
    #nocars_directory = '/Users/kennt/classes/udacity/small/non-vehicles_smallset/*/*.jpeg'

    for image in glob.iglob(cars_directory):
        cars.append(image)
    for image in glob.iglob(nocars_directory):
        notcars.append(image)

    print("Found ", len(cars), " vehicle images")
    print("Found ", len(notcars), " non-vehicle images")

    # shuffle the list of cars
    np.random.shuffle(cars)
    np.random.shuffle(notcars)

    cars = cars[0:len(cars)//3]
    notcars = notcars[0:len(notcars)//3]


    ### TODO: Tweak these parameters and see how the results change.
    #color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    #orient = 9  # HOG orientations
    #pix_per_cell = 16 # HOG pixels per cell
    #cell_per_block = 2 # HOG cells per block
    #hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    #spatial_size = (16, 16) # Spatial binning dimensions
    #hist_bins = 8    # Number of histogram bins
    #spatial_feat = True # Spatial features on or off
    #hist_feat = True # Histogram features on or off
    #hog_feat = True # HOG features on or off
    y_start_stop = [400, None] # Min and max in y to search in slide_window()

    car_features = extract_features(cars, **features_params)
    notcar_features = extract_features(notcars, **features_params)

    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(
        scaled_X, y, test_size=0.2, random_state=rand_state)

    print('Training set size: ', len(X_train))
    print('Test set size: ', len(X_test))

    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))

    t=time.time()

    C_range = [0.5, 1, 1.5, 2, 3, 4, 5]
    param_grid = dict(C=C_range)
    svc = GridSearchCV(svm.SVC(), param_grid, verbose=1)
    svc.fit(X_train, y_train)
    print(svc.best_params_)
    print(svc.best_score_)

    # Use a linear SVC
    #svc = LinearSVC(C=2)
    #svc.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    return svc, X_scaler



In [6]:
y_start_stop = (400, None)

svc, X_scaler = train_svc(**features_params)

Found  8792  vehicle images
Found  8968  non-vehicle images
Training set size:  4735
Test set size:  1184
Using: 11 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 2052
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:  4.6min finished


{'C': 2}
0.990285110876
287.93 Seconds to train SVC...
Test Accuracy of SVC =  0.9916


## 6. Sub-sampling HOG

In [7]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler,
              color_space='RGB',
              spatial_size=(32, 32),
              hist_bins=32,
              orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0,
              spatial_feat=True, hist_feat=True, hog_feat=True):

    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_colorspace(img_tosearch, 'RGB', color_space)

    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog_array = [None]*4
    if hog_feat is True and (hog_channel == 1 or hog_channel == 'ALL'):
        hog_array[1] = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    if hog_feat is True and (hog_channel == 2 or hog_channel == 'ALL'):
        ch2 = ctrans_tosearch[:,:,1]
        hog_array[2] = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    if hog_feat is True and (hog_channel == 1 or hog_channel == 'ALL'):
        ch3 = ctrans_tosearch[:,:,2]
        hog_array[3] = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    bboxes = []
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

            img_features = []
            
            # Get color features
            if spatial_feat is True:
                spatial_features = bin_spatial(subimg, size=spatial_size)
                img_features.append(spatial_features)
            
            if hist_feat is True:
                hist_features = color_hist(subimg, nbins=hist_bins)
                img_features.append(hist_features)

            # Extract HOG for this patch
            if hog_feat is True:
                if hog_channel == 'ALL':
                    hog_features = []
                    for channel in range(1,4):
                        img_features.append(hog_array[channel][ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel())
                else:
                    img_features.append(hog_array[hog_channel][ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel())

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack(img_features).reshape(1, -1))
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                bboxes.append([(xbox_left, ytop_draw+ystart),
                               (xbox_left+win_draw, ytop_draw+win_draw+ystart)])
                
    return bboxes



## 7. Heatmap and Labeling

In [8]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
        
    # Reinforce the hotspots
    for box in bbox_list:
        # Add += 2 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        if np.max(heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]]) > 1:
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 2

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,255,0), 6)
    # Return the image
    return img





## 8. VehicleFinder Class

In [17]:
class VehicleFinder:
    def __init__(self, heatmap_threshold=9, heatmap_window=4):
        self.stats = dict()
        
        # Take an average of the heatmap
        self.heatmap_threshold = heatmap_threshold
        self.average_heatmap = None
        self.heatmaps = collections.deque(list(), heatmap_window)
        self.heatmap_window = heatmap_window
        
        self.average_heatmap = None
    
    def process_image(self, img):
        """
        Takes in an image (8-bit colors) and returns an image.
        """
        frame = np.copy(img)
        frame = frame.astype(np.float32)/255

        bboxes = []
        bboxes.extend(find_cars(frame, 400, 650, 0.8, svc, X_scaler, **features_params))
        bboxes.extend(find_cars(frame, 400, 720, 1.5, svc, X_scaler, **features_params))
        bboxes.extend(find_cars(frame, 400, 720, 2.0, svc, X_scaler, **features_params))

        # Initialize the image totals
        if self.average_heatmap is None:
            self.average_heatmap = np.zeros_like(frame[:,:,0]).astype(np.float)
            heat = np.ones_like(frame[:,:,0]).astype(np.float)*0.3
            for _ in range(self.heatmap_window):
                self.average_heatmap += heat

        # generate the heatmap for this image
        heat = np.zeros_like(frame[:,:,0]).astype(np.float)
        heat = add_heat(heat, bboxes)

        if self.heatmaps.maxlen == len(self.heatmaps):
            self.average_heatmap -= self.heatmaps[0]
        self.average_heatmap += heat
        self.heatmaps.append(heat)

        heatmap = np.copy(self.average_heatmap)
        heatmap = apply_threshold(heatmap, self.heatmap_threshold)    

        labels = label(heatmap)

        return draw_labeled_bboxes(img, labels)



## 8. Video Pipeline

In [18]:
carfinder = VehicleFinder(heatmap_window=4, heatmap_threshold=9)

test_output = "./project_output.mp4"
clip1 = VideoFileClip('./project_video.mp4')
test_clip1 = clip1.fl_image(carfinder.process_image)
%time test_clip1.write_videofile(test_output, audio=False)
print(carfinder.stats)

[MoviePy] >>>> Building video ./project_output.mp4
[MoviePy] Writing video ./project_output.mp4


100%|█████████▉| 1260/1261 [37:19<00:01,  1.86s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_output.mp4 

CPU times: user 36min 28s, sys: 52.6 s, total: 37min 21s
Wall time: 37min 20s
{}
